In [2]:
%matplotlib inline
#%set_env PYTORCH_ROCM_ARCH="gfx1100"
#%set_env HSA_OVERRIDE_GFX_VERSION=10.3.0
#%set_env HIP_VISIBLE_DEVICES=0
#%set_env ROCM_PATH=/opt/rocm
#%set_env HIP_LAUNCH_BLOCKING=1
%set_env AMD_LOG_LEVEL=3

env: AMD_LOG_LEVEL=3


In [ ]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

manual_seed = 9999
# manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)
torch.use_deterministic_algorithms(True) # for reproducible results

In [ ]:
dataroot = "data/celeba"
workers = 2
batch_size = 128
image_size = 64
nc = 3                   # num of color channels
nz = 100                 # len of latent vector (size of generator input)
ngf = 64                 # size of feature maps in generator
ndf = 64                 # size of feature maps in discriminator
num_epochs = 5
lr = 0.0002              # learning rate
beta1 = 0.5              # beta1 hyperparameter for Adam optimizers

# num of GPUs (0 == run in CPU mode)
ngpu = 1 if torch.cuda.is_available() else 0

In [ ]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=batch_size,
                                         shuffle=True,
                                         num_workers=workers)

print(torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)
print(torch.cuda.is_available())
#print(torch.rand(3, 3).to(device))
#device = torch.device("cpu")

real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")

plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64],
                                         padding=2,
                                         normalize=True).cpu(),
                        (1,2,0)))

#plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            #  input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    
    def forward(self, input):
        return self.main(input)

In [ ]:
netG = Generator(ngpu).to(device)

# For multi-GPU
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly
# apply all weights to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

print(netG)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # Input is ``(nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netD = Discriminator(ngpu).to(device)

# For multi-GPU
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallell(netD, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly
# initialize all weights like this:
# ``to mean=0, stdev=0.2``.
netD.apply(weights_init)

print(netD)

In [ ]:
criterion = nn.BCELoss()

# Create a batch of latent vectors that we will use to
# visualize the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        """
            1) Update D network:
            maximize log(D(x)) + log(1 - D(G(z))
        """
        ## train w/ all-real batch
        netD.zero_grad()
        # format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,),
                           real_label,
                           dtype=torch.float,
                           device=device
                          )
        # format pass real batch thru D
        output = netD(real_cpu).view(-1)
        # Calc loss on all-real batch
        errD_real = criterion(output, label)
        # Calc gradients for D in backwards pass
        errD_real.backward()
        D_x = output.mean().item()

        ## train w/ all-fake batch
        # Generate batch of latent noise
        noise = torch.randn(b_size,
                            nz, 
                            1,
                            1,
                            device=device
                           )
        # Generate fake img batch w/ G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch w/ D
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch,
        # accumulated w/ prev gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake
        # and real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        """
            2) Update G Network
            maximize log(D(G(z)))
        """
        netG.zero_grad()
        # real labels are real for generator cost
        label.fill_(real_label)
        # since we just updated D, 
        # perform another forward pass of all-fake
        # batch thru D
        output = netD(fake).view(-1)
        # Calc G's loss based on this output
        errG = criterion(output, label)
        # Calc gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLossG: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by 
        # saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs - 1) \
                                  and (i == len(dataloader) - 1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1
            